#Setup

In [ ]:
!pip install -q pyspark
!pip install -q vaderSentiment
!pip install -q tweet-preprocessor
!pip install -q git+https://github.com/tweepy/tweepy.git #Updated version allows usage of API.search_tweets attribute
!pip install -q yfinance

In [ ]:
import pyspark as spark 
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, udf, monotonically_increasing_id, unix_timestamp, round, avg
import yfinance as yf
import pandas as pd
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')
import re
import tweepy
from tweepy import OAuthHandler
import pandas as pd
import time
import json
import csv
import preprocessor as p #cleaning each tweet using tweet-preprocessor like removing hashtags,urls,emojis....
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime 
from dateutil import parser
from google.colab import drive 
sc= spark.SparkContext()
sql= spark.SQLContext(sc)
import numpy as np
from datetime import date, timedelta
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Functions

In [ ]:
access_token=""
access_token_secret=""
consumer_key="
consumer_secret=""
API_Data = access_token, access_token_secret, consumer_key, consumer_secret

def get_tweets_to_csv(API_Data, count, text_query, FileName):
    auth = tweepy.OAuthHandler(API_Data[2], API_Data[3])
    auth.set_access_token(API_Data[0], API_Data[1])
    api = tweepy.API(auth, wait_on_rate_limit=True)
    df_query_based_tweets = pd.DataFrame()
    try:
        tweets = tweepy.Cursor(api.search_tweets,q=text_query).items(count)
        for tweet in tweets:
            df_query_based_tweets = df_query_based_tweets.append(
                              {'Created at' : tweet._json['created_at'],
                                           'User ID': tweet._json['id'],
                                  'User Name': tweet.user._json['name'],
                                            'Text': tweet._json['text'],
                         'Description': tweet.user._json['description'],
                               'Location': tweet.user._json['location'],
                 'Followers Count': tweet.user._json['followers_count'],
                     'Friends Count': tweet.user._json['friends_count'],
                   'Statuses Count': tweet.user._json['statuses_count'],
             'Profile Image Url': tweet.user._json['profile_image_url'],}, ignore_index=True)
    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(3)
    df_query_based_tweets.to_csv(FileName+'.csv', index=False)
    return

def pull_csv(FileName):
    return(pd.read_csv(FileName+'.csv'))

def function_udf(input_str):
  """Takes string inputs and outputs cleaned strings - runs over tweets"""
  input_str = re.sub(r'RT', '', input_str)
  p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION)
  input_str = p.clean(input_str)
  return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", input_str).split())

def senti_score_udf(sentence):
  """Takes tweet string and performs sentiment analysis. Makes 3 new columns 
  positive, negative and neutral in range 0-1."""
  snt = analyser.polarity_scores(sentence)
  return ([snt['neg'], snt['neu'], snt['pos'], snt['compound']])


def Tw_Time_format(stri):  #manipulating and casting the strings(DateTime) of tweets dataframe to timestamps
  """Takes the time column and outputs datetime-readable time signature"""
  datetime_object = datetime.strptime(stri, '%a %b %d %I:%M:%S %z %Y')
  #dic = {'Dec':'11','Oct':'10'}
  #ans = ''
  #ans += stri[-4:]+'-'+ dic[stri[4:7]]+'-'+stri[8:19]
  return str(datetime_object)

def Btc_Time_format(input_str): #manipulating and casting the strings(DateTime) of BTC dataframe to timestamps
    input_str = re.sub(r'/17','', input_str)
    input_str = '2017-'+ input_str
    input_str = re.sub(r'/', '-', input_str)
    input_str += ':00'
    return input_str[:10]+""+input_str[10:]

def create_spark_dataframe(file_name):
   """
   will return the spark dataframe input pandas dataframe - 
   https://stackoverflow.com/questions/39862211/typeerror-when-converting-pandas-to-spark/40344485
   """
   pandas_data_frame = pd.read_csv(file_name, converters= {"PRODUCT": str})
   for col in pandas_data_frame.columns:
    if ((pandas_data_frame[col].dtypes != np.int64) & (pandas_data_frame[col].dtypes != np.float64)):
      pandas_data_frame[col] = pandas_data_frame[col].fillna('')

   spark_data_frame = sql.createDataFrame(pandas_data_frame)
   return spark_data_frame


#Get Data

In [ ]:
#Previous:
#btc= pd.read_csv("gdrive/My Drive/BitCoinPrice.csv", error_bad_lines=False, engine='python', header= None)
#twitter=pd.read_csv('gdrive/My Drive/tweetsfinal2.csv',error_bad_lines=False,engine = 'python',header = None)

#New:
tweet_count = 10000
FileName = "TweetsBtc"
get_tweets_to_csv(API_Data, tweet_count, ['BTC','Bitcoin', 'bitcoin'], FileName)

Rate limit reached. Sleeping for: 838
Rate limit reached. Sleeping for: 833
Rate limit reached. Sleeping for: 823


failed on_status, 


#Tweet Data processing

In [ ]:
FullDataTwitter = create_spark_dataframe((FileName+".csv"))

FullDataTwitter.select(monotonically_increasing_id().alias("rowId"), "*")
FullDataTwitter= FullDataTwitter.withColumnRenamed('0', 'DateTime')#Setting column names to the twitter dataset
FullDataTwitter=FullDataTwitter.withColumnRenamed('1', 'Tweet')

Tw_samp= FullDataTwitter

##Cleaning Tweet Data

In [ ]:
func_udf = udf(function_udf, StringType())
CleanDF = Tw_samp.withColumn('CleanedTweets', func_udf(Tw_samp['text']))

##Sentiment Analysis

In [ ]:
analyser = SentimentIntensityAnalyzer()
func_udf2 = udf(senti_score_udf, ArrayType(FloatType()))
CleanDF = CleanDF.withColumn('p_neg', func_udf2(CleanDF['CleanedTweets'])[0])
CleanDF = CleanDF.withColumn('p_neu', func_udf2(CleanDF['CleanedTweets'])[1])
CleanDF = CleanDF.withColumn('p_pos', func_udf2(CleanDF['CleanedTweets'])[2])
CleanDF = CleanDF.withColumn('p_comp', func_udf2(CleanDF['CleanedTweets'])[3])

##Cleaning Time Column

In [ ]:
func_udf3 = udf(Tw_Time_format,StringType())
CleanDF = CleanDF.withColumn('DateTime_c', func_udf3(CleanDF['Created at']))
CleanDF = CleanDF.withColumn("DateTime_casted",CleanDF['DateTime_c'].cast(TimestampType()))

FinalTw = CleanDF.selectExpr("DateTime_casted as Date_Time", "CleanedTweets as Cleaned_Tweets", "p_neg","p_neu","p_pos","p_comp")

#Asset Processing

In [ ]:

data = yf.download('BTC-USD', start="2021-01-12")['Adj Close'][0]
pandasDF = FinalTw.toPandas()
asset_price = []
for i in range(len(pandasDF)):
  asset_price.append(yf.download('BTC-USD', start=pandasDF['Date_Time'][i])['Adj Close'][0])

[*********************100%***********************]  1 of 1 completed


#Completed

In [ ]:
pandasDF

,Date_Time,Cleaned_Tweets,p_neg,p_neu,p_pos,p_comp,Asset_Price
0,2021-11-01 17:37:03,Bitcoin BTC Do you see it,0.000,1.000,0.000,0.0000,61004.40625
1,2021-11-01 17:37:03,This is Revolotto RVL Revolutionary Lottery Co...,0.000,0.891,0.109,0.2960,61004.40625
2,2021-11-01 17:37:03,Bitcoin fails worst case scenario monthly clos...,0.347,0.653,0.000,-0.7845,61004.40625
3,2021-11-01 17:37:02,Current Bitcoin Price is 58474 BTC Crypto,0.000,1.000,0.000,0.0000,61004.40625
4,2021-11-01 17:37:00,Ethereum hits 3 1 2 year high against Bitcoin ...,0.000,1.000,0.000,0.0000,61004.40625
...,...,...,...,...,...,...,...
95,2021-11-01 17:35:33,bitcoin btc crypto ico eth altcoin smash smash...,0.000,1.000,0.000,0.0000,61004.40625
96,2021-11-01 17:35:32,PAICIPATE IN BITBULL TRADING COMPETITION AND A...,0.000,0.600,0.400,0.8647,61004.40625
97,2021-11-01 17:35:30,This is Revolotto RVL Revolutionary Lottery Co...,0.000,0.891,0.109,0.2960,61004.40625
98,2021-11-01 17:35:29,Imagine an AI that learns to code and writes a...,0.000,0.856,0.144,0.4939,61004.40625
